In [ ]:
!pip install -U sentence-transformers

In [ ]:
!pip install tweet-preprocessor

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import re
import preprocessor as p
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [ ]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
df = pd.read_csv('topical_chat.csv')
df=df.head(70000)
df.head()

,conversation_id,message,sentiment
0,1,Are you a fan of Google or Microsoft?,Curious to dive deeper
1,1,Both are excellent technology they are helpfu...,Curious to dive deeper
2,1,"I'm not a huge fan of Google, but I use it a...",Curious to dive deeper
3,1,Google provides online related services and p...,Curious to dive deeper
4,1,"Yeah, their services are good. I'm just not a...",Curious to dive deeper


In [ ]:
unique_values = df['sentiment'].unique()
print(unique_values)

[' Curious to dive deeper' ' Happy' ' Neutral' ' Surprised' ' Disgusted'
 ' Sad' ' Fearful' ' Angry']


In [ ]:
l=[]
for i in df['sentiment']:
  if i==' Curious to dive deeper':
     l.append(0)
  elif i==' Happy':
     l.append(1)
  elif i==' Neutral':
     l.append(2)
  elif i==' Surprised':
     l.append(3)
  elif i==' Disgusted':
     l.append(4)
  elif i==' Sad':
     l.append(5)
  elif i==' Fearful':
     l.append(6)
  elif i==' Angry':
     l.append(7)


In [ ]:
df['cluster']=l

In [ ]:
import pandas as pd

# Assuming df is your DataFrame
shuffled_df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
def clean_tweets(df):
    # punctuations we want to be replaced
    REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
    REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")
    corpus = []
    for line in df:
        # clean data by sending it to tweet_processor
        sentence = p.clean(line)
        # remove puctuation
        sentence = REPLACE_NO_SPACE.sub("", sentence.lower()) # convert all tweets to lower cases
        sentence = REPLACE_WITH_SPACE.sub(" ", sentence)
        corpus.append(sentence)
    return corpus

In [ ]:
df['message_clean'] = clean_tweets(df['message'])
df.shape

(70000, 5)

In [ ]:
df_sample = df.groupby('cluster').head(5000)

In [ ]:
corpus = list(df_sample['message'])

In [ ]:
trueValues=df_sample['cluster']

In [ ]:
import numpy as np
corpus_embeddings = embedder.encode(corpus)
corpus_embeddings=corpus_embeddings /  np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)
corpus_embeddings

array([[-0.09441944, -0.06091456,  0.03195329, ..., -0.01119272,
         0.03922331,  0.00803194],
       [-0.0887982 ,  0.01459854, -0.05495671, ..., -0.00030676,
         0.03732098,  0.02115922],
       [-0.12374243, -0.13280554,  0.01056302, ...,  0.02783513,
         0.03091876,  0.06489429],
       ...,
       [-0.02478255,  0.09410773,  0.04287953, ..., -0.09710478,
         0.02238145, -0.06050058],
       [ 0.04262629,  0.0187572 ,  0.0830884 , ..., -0.13221239,
         0.03434359, -0.01935124],
       [ 0.00298768, -0.00138212, -0.02701899, ..., -0.11132045,
        -0.07380995, -0.03206391]], dtype=float32)

In [ ]:
num_clusters = 8
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
ll=list(cluster_assignment)
ll=list((i,j) for i,j in zip(trueValues,ll))
ll

In [ ]:
trueValues

0        0
1        0
2        0
3        0
4        0
        ..
69695    5
69723    5
69744    5
69757    5
69893    6
Name: cluster, Length: 22134, dtype: int64

In [ ]:
cluster_df = pd.DataFrame(corpus, columns = ['corpus'])
cluster_df['cluster'] = cluster_assignment
cluster_df.head()

,corpus,cluster
0,Are you a fan of Google or Microsoft?,5
1,Both are excellent technology they are helpfu...,5
2,"I'm not a huge fan of Google, but I use it a...",5
3,Google provides online related services and p...,5
4,"Yeah, their services are good. I'm just not a...",5


In [ ]:
clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(corpus[sentence_id])

for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster)
    print("")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
unique_predicted_labels = np.unique(cluster_assignment)

print(unique_predicted_labels)

[0 1 2 3 4 5 6 7]


In [ ]:
import numpy as np

def tri_et_index(tableau):
    # Obtenir les indices des valeurs triées en ordre décroissant
    indices_tries = np.flip(np.argsort(tableau))

    return indices_tries

In [ ]:
import numpy as np

def map_labels(predicted_labels, true_labels):
    unique_predicted_labels = np.unique(predicted_labels)
    label_mapping = {index: 10 for index in unique_predicted_labels}

    for predicted_label in unique_predicted_labels:
        i=0
        mask = predicted_labels == predicted_label
        cluster_true_labels = true_labels[mask]
        true_label_counts = np.bincount(cluster_true_labels)
        indexs=tri_et_index(true_label_counts)
        most_frequent_true_label=indexs[i]
        l=[label_mapping[label] for label in unique_predicted_labels]

        while most_frequent_true_label in l[:predicted_label]:
            most_frequent_true_label = indexs[i+1]
            i+=1
        label_mapping[predicted_label] = most_frequent_true_label

    return label_mapping

label_mapping = map_labels(cluster_assignment, trueValues)



In [ ]:
mapped_predicted_labels = np.array([label_mapping[label] for label in cluster_assignment])
unique_values = np.unique(mapped_predicted_labels)
label_mapping

{0: 3, 1: 1, 2: 2, 3: 0, 4: 5, 5: 4, 6: 7, 7: 6}

In [ ]:
from sklearn.metrics import accuracy_score
clustering_accuracy = accuracy_score(trueValues, mapped_predicted_labels)

clustering_accuracy

0.11457486220294569